# Designing the Controller

In this notebook, the following steps will be taken to create a controller for the CMG
* Linearize the system
* Find the eigenvalues of F for different values of K
* Pick value of K that gives negative eigenvalues for F and test in CMG

In [2]:
#importing the required libraries
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy import linalg

## Linearizing the System

In [8]:
#All the required symbols
q1,q2,v1,v2,v3,tau2,tau3 = sym.symbols('q1,q2,v1,v2,v3,tau2,tau3')

In [11]:
#Put the F function here
f = sym.Matrix([[v1],[-(5*(200*tau3*sym.sin(q2)+sym.sin(2*q2)*v1*v2+2*sym.cos(q2)*v2*v3)/(10*(sym.sin(q2))**2 -511))],[v2],[10/11 * (100*tau2 - sym.cos(q2)*v1*v3)],[-((51100*tau3+5*sym.sin(2*q2)*v2*v3+511*sym.cos(q2)*v1*v2)/(10*(sym.sin(q2))**2 -511))]])


Matrix([
[                                                                               v1],
[-(1000*tau3*sin(q2) + 5*v1*v2*sin(2*q2) + 10*v2*v3*cos(q2))/(10*sin(q2)**2 - 511)],
[                                                                               v2],
[                          90.9090909090909*tau2 - 0.909090909090909*v1*v3*cos(q2)],
[      -(51100*tau3 + 511*v1*v2*cos(q2) + 5*v2*v3*sin(2*q2))/(10*sin(q2)**2 - 511)]])

In [14]:
#Making f an executable function
f_num = sym.lambdify((q1,q2,v1,v2,v3,tau2,tau3), f)


<function _lambdifygenerated(q1, q2, v1, v2, v3, tau2, tau3)>

In [15]:
#Equilibrium points
#q1 is not in any of the equations, so it can be anything, let's make it about 45 degrees (desired angle)
#v3 can also be anything, but since we're starting off at 100 rpm, I'll set it as that.
q1_e = 45*np.pi/180
q2_e = 0
v1_e = 0
v2_e = 0
v3_e = 10.472 #100 rpm, starting v3
tau2_e = 0
tau3_e = 0

In [18]:
#checking if the equilibrium points work:
f_eq = f_num(q1_e,q2_e,v1_e,v2_e,v3_e,tau2_e,tau3_e)
f_eq #evaluates to 0! Nice!

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [27]:
#Now we find the Jacobians
f_jacob_x = f.jacobian([q1,q2,v1,v2,v3])
f_jacob_u = f.jacobian([tau2,tau3])

#And then we find functions for A and B using this jacobian
A_num = sym.lambdify((q1,q2,v1,v2,v3,tau2,tau3),f_jacob_x)
B_num = sym.lambdify((q1,q2,v1,v2,v3,tau2,tau3),f_jacob_u)

#Finally, we find the linearized state space model by evaluating A_num and B_num at the equilibrium points
A = A_num(q1_e,q2_e,v1_e,v2_e,v3_e,tau2_e,tau3_e).astype(float)
B = B_num(q1_e,q2_e,v1_e,v2_e,v3_e,tau2_e,tau3_e).astype(float)

With all these steps, we have linearized our equations of motions about the chosen equilibrium point to create a state space model.

In [31]:
A

array([[ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.        ,  0.20493151, -0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        , -9.52      ,  0.        , -0.        ],
       [ 0.        ,  0.        , -0.        ,  0.        , -0.        ]])

In [32]:
B

array([[  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [ 90.90909091,   0.        ],
       [  0.        , 100.        ]])

## Finding Stable Controller Gains

In [53]:
K = np.zeros((2,5)) #This is our controller gains, we need to test different values
K = np.array([[4,5,4,4,0],[0,0,0,0,12]])

In [55]:
F  = A - B@K
s = linalg.eigvals(F)
s.real

array([-3.62352244e+02, -6.42059642e-01, -6.42059642e-01,  0.00000000e+00,
       -1.20000000e+03])

In [52]:
x0 = sym.Matrix([[0],[0],[0],[0],[10.472]])
F_sym = sym.Matrix(F)
t = sym.Symbol('t')
x_sym = sym.exp(F_sym*t)@x0

KeyboardInterrupt: 